In [1]:
from pref_voting.profiles_with_ties import *
from pref_voting.voting_methods import *
from pref_voting.analysis import *
from pref_voting.generate_profiles import *
from pref_voting.generate_weighted_majority_graphs import *

from pref_voting.utility_profiles import UtilityProfile, write_utility_profiles_to_json, read_utility_profiles_from_json
from pref_voting.rankings import Ranking
from pref_voting.generate_utility_profiles import *
from pref_voting.generate_utility_profiles import *
from pref_voting.utility_methods import *

from pref_voting.spatial_profiles import SpatialProfile
from pref_voting.generate_spatial_profiles import *
from pref_voting.utility_functions import *
from pref_voting.probabilistic_methods import *
from tqdm.notebook import tqdm
import nashpy as nash
import numpy as np
import random2 as random
from pref_voting.mappings import _Mapping
from multiprocess import Pool, cpu_count, current_process
from numba import njit, float32
import pickle
import json
from pref_voting.monotonicity_axioms import *
from pref_voting.helper import *
from pref_voting.variable_candidate_axioms import *
from pref_voting.grade_methods import *
from pref_voting.mappings import *

In [2]:
def has_strong_path_dfs(A, source, target, k):
    """Given a square matrix A, return True if there is a path from source to target in the associated directed graph     where each edge has a weight greater than or equal to k, and False otherwise."""
    
    n = A.shape[0] # assume A is a square matrix
    visited = np.zeros(n, dtype=bool)

    def dfs(node):
        if node == target:
            return True
        visited[node] = True
        for neighbor, weight in enumerate(A[node, :]):
            if weight >= k and not visited[neighbor]:
                if dfs(neighbor):
                    return True
        return False

    return dfs(source)


def has_strong_path_bfs(matrix, source, target, k):
    '''
    Given a square `matrix`, return `True` if there is a path from
    `source` to `target` in the associated directed graph, where each
    edge has a weight greater than or equal to `k`, and `False`
    otherwise.
    '''
    n = matrix.shape[0]  # `A` is square
    # keep track of visited nodes (initially all `False`)
    visited = np.zeros(n, dtype=bool)
    visited[source] = True  # do not revisit the `source` node

    def bfs(nodes):
        '''
        Breadth-first search implementation:
        Search starting from `nodes` in `matrix` until a path to
        `target` is found or until all nodes are searched. Since 
        Condorcet cycles are exceedingly rare in real elections and
        typically do not involve many candidates[1], a breadth-first
        search of the margins graph will be fastest to detect such a
        cycle.
        [1] (Gehrlein and Lepelley, "Voting Paradoxes and Group
            Coherence")
        '''
        queue = []  # nodes to search next cycle

        for node in nodes:
            # check for a direct path from `node` to `target`
            if matrix[node, target] >= k:
                return True

            # queue neighbors to check for a path to `target`
            visited[node] = True
            for neighbor, weight in enumerate(matrix[node, :]):
                if weight >= k and not visited[neighbor]:
                    queue.append(neighbor)

        return bfs(queue) if queue else False

    return bfs([source])


In [3]:
@vm(name="Split Cycle DFS")
def split_cycle_dfs(edata, curr_cands = None, strength_function = None):

    
    strength_matrix, cand_to_cindex = edata.strength_matrix(curr_cands = curr_cands, strength_function=strength_function)

    candidates = edata.candidates if curr_cands is None else curr_cands  

    strength_function = edata.margin if strength_function is None else strength_function 

    potential_winners = set(candidates)

    for a in candidates:
        for b in candidates:
            if strength_function(b,a) > 0 and not has_strong_path_dfs(strength_matrix, cand_to_cindex(a), cand_to_cindex(b), strength_function(b,a)):
                potential_winners.discard(a)
                break

    return sorted(potential_winners)


@vm(name="Split Cycle BFS")
def split_cycle_bfs(edata, curr_cands = None, strength_function = None):

    
    strength_matrix, cand_to_cindex = edata.strength_matrix(curr_cands = curr_cands, strength_function=strength_function)

    candidates = edata.candidates if curr_cands is None else curr_cands  

    strength_function = edata.margin if strength_function is None else strength_function 

    potential_winners = set(candidates)

    for a in candidates:
        for b in candidates:
            if strength_function(b,a) > 0 and not has_strong_path_bfs(strength_matrix, cand_to_cindex(a), cand_to_cindex(b), strength_function(b,a)):
                potential_winners.discard(a)
                break

    return sorted(potential_winners)



In [4]:

mgs = [generate_profile(10, 20).margin_graph() for _ in range(1000)]



In [5]:
%%time

[split_cycle_dfs(mg) for mg in mgs]


CPU times: user 58.4 ms, sys: 1.59 ms, total: 59.9 ms
Wall time: 59.2 ms


[[3, 5],
 [3],
 [1, 8, 9],
 [3],
 [6, 7],
 [3],
 [3],
 [8],
 [4, 5],
 [0],
 [3, 5, 7],
 [2, 4],
 [5],
 [2, 3, 9],
 [8],
 [1, 2, 9],
 [0, 7],
 [5],
 [1, 2, 3, 7, 9],
 [1],
 [5, 6],
 [3, 6],
 [1],
 [1, 2, 8, 9],
 [9],
 [2],
 [3, 4, 6, 7],
 [5],
 [5],
 [4, 7],
 [3],
 [7, 9],
 [1],
 [6, 8, 9],
 [0],
 [0, 4, 6],
 [5, 7],
 [6],
 [7],
 [1],
 [0, 4],
 [4, 7],
 [3],
 [1],
 [1, 7],
 [2],
 [2],
 [0],
 [2],
 [2],
 [5, 9],
 [0, 3, 7, 8],
 [7],
 [1],
 [1, 2, 7, 9],
 [1, 3],
 [5],
 [6],
 [1, 3, 9],
 [4, 5],
 [4],
 [9],
 [0, 2],
 [1, 6, 7, 8],
 [6],
 [3, 5],
 [1, 2, 9],
 [5],
 [7],
 [4],
 [5],
 [0, 2],
 [8],
 [0, 9],
 [8],
 [2],
 [2],
 [7],
 [2, 5],
 [5],
 [0, 4, 6],
 [3],
 [4, 8],
 [4, 6],
 [7],
 [3],
 [1],
 [3],
 [7],
 [9],
 [2, 7, 9],
 [0, 1],
 [2, 4],
 [0, 2, 7, 9],
 [0, 6, 8],
 [5],
 [3, 4],
 [0, 7],
 [7],
 [7],
 [4],
 [6],
 [1, 7],
 [2, 7],
 [0, 5],
 [2, 3],
 [6],
 [3, 5],
 [3, 7],
 [2, 6],
 [9],
 [7],
 [0, 4, 7, 8],
 [5],
 [3],
 [3],
 [0, 4, 6],
 [2, 9],
 [4],
 [4, 8],
 [1],
 [5],
 [3],
 [3],
 

In [6]:
%%time
[split_cycle_bfs(mg) for mg in mgs]

CPU times: user 79.2 ms, sys: 1.96 ms, total: 81.2 ms
Wall time: 88.1 ms


[[3, 5],
 [3],
 [1, 8, 9],
 [3],
 [6, 7],
 [3],
 [3],
 [8],
 [4, 5],
 [0],
 [3, 5, 7],
 [2, 4],
 [5],
 [2, 3, 9],
 [8],
 [1, 2, 9],
 [0, 7],
 [5],
 [1, 2, 3, 7, 9],
 [1],
 [5, 6],
 [3, 6],
 [1],
 [1, 2, 8, 9],
 [9],
 [2],
 [3, 4, 6, 7],
 [5],
 [5],
 [4, 7],
 [3],
 [7, 9],
 [1],
 [6, 8, 9],
 [0],
 [0, 4, 6],
 [5, 7],
 [6],
 [7],
 [1],
 [0, 4],
 [4, 7],
 [3],
 [1],
 [1, 7],
 [2],
 [2],
 [0],
 [2],
 [2],
 [5, 9],
 [0, 3, 7, 8],
 [7],
 [1],
 [1, 2, 7, 9],
 [1, 3],
 [5],
 [6],
 [1, 3, 9],
 [4, 5],
 [4],
 [9],
 [0, 2],
 [1, 6, 7, 8],
 [6],
 [3, 5],
 [1, 2, 9],
 [5],
 [7],
 [4],
 [5],
 [0, 2],
 [8],
 [0, 9],
 [8],
 [2],
 [2],
 [7],
 [2, 5],
 [5],
 [0, 4, 6],
 [3],
 [4, 8],
 [4, 6],
 [7],
 [3],
 [1],
 [3],
 [7],
 [9],
 [2, 7, 9],
 [0, 1],
 [2, 4],
 [0, 2, 7, 9],
 [0, 6, 8],
 [5],
 [3, 4],
 [0, 7],
 [7],
 [7],
 [4],
 [6],
 [1, 7],
 [2, 7],
 [0, 5],
 [2, 3],
 [6],
 [3, 5],
 [3, 7],
 [2, 6],
 [9],
 [7],
 [0, 4, 7, 8],
 [5],
 [3],
 [3],
 [0, 4, 6],
 [2, 9],
 [4],
 [4, 8],
 [1],
 [5],
 [3],
 [3],
 